In [6]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
import tkinter.messagebox
import json
from tkinter import Scrollbar
from tkinter import *
from tkinter.ttk import Treeview
from tkinter.messagebox import showinfo
from tkinter import filedialog


global df


# 重设索引，并按照差距排序
def reset_and_sort_index(df):
    df.reset_index(drop=True, inplace=True)
    df.sort_values(by=['差距'], ascending=True, inplace=True)
    return df

# 师范、医科、军校的过滤功能
def filter_school_types(df):
    selected_rows = df.copy()

    # 判断是否勾选了只看师范、医科、军校选项，如果勾选了，则取消其他选项的勾选状态
    if filter_normal_var.get():
        filter_medical_var.set(False)
        filter_military_var.set(False)
        filter_var.set(False)  # 取消"过滤师范、医科、军校"的勾选状态

        # 应用只看师范过滤选项
        selected_rows = selected_rows[selected_rows['学院名称'].str.contains('师范')]

    elif filter_medical_var.get():
        filter_normal_var.set(False)
        filter_military_var.set(False)
        filter_var.set(False)  # 取消"过滤师范、医科、军校"的勾选状态

        # 应用只看医科过滤选项
        selected_rows = selected_rows[selected_rows['学院名称'].str.contains('医科|中医药|药科|医学')]

    elif filter_military_var.get():
        filter_normal_var.set(False)
        filter_medical_var.set(False)
        filter_var.set(False)  # 取消"过滤师范、医科、军校"的勾选状态

        # 应用只看军校过滤选项
        selected_rows = selected_rows[selected_rows['学院名称'].str.contains('军校|军校|海军|解放军|陆军|火箭军|海军|军医|空军|武警|战略')]

    else:
        # 应用学校类型过滤选项
        filter_enabled = filter_var.get()
        if filter_enabled:
            selected_rows = selected_rows[~selected_rows['学院名称'].str.contains("师范|医科|中医药|药科|医学|军校|海军|解放军|陆军|火箭军|海军|军医|空军|武警|战略")]

    return selected_rows

# 根据专业查询的功能
def filter_by_major(row, user_major):
    major_name = row['专业名称']
    return user_major.lower() in major_name.lower()

# 主要功能，包含总分查询、政治、英语单独查询、学校与专业查询
def find_closest_scores_with_major():
    global df
    global selected_regions
    
    # 根据selected_regions列表中的地区名称读取对应的数据文件
    dfs = []
    if selected_regions:  # 如果selected_regions不为空
        for region in selected_regions:
            file_path = config_data[region]
            df = pd.read_excel(file_path)
            dfs.append(df)
    else:  # 如果selected_regions为空，就读取所有地区的数据
        for region, file_path in config_data.items():
            df = pd.read_excel(file_path)
            dfs.append(df)
    df = pd.concat(dfs)
    
    selected_rows = filter_school_types(df)
    
    user_score = int(float(entry_score.get())) if entry_score.get() else None
    user_major = entry_major.get().strip() if entry_major.get() else None
    float_value = int(float(entry_float.get())) if entry_float.get() else 20

    politics_score = int(float(entry_politics.get())) if entry_politics.get() else None
    english_score = int(float(entry_english.get())) if entry_english.get() else None

    if politics_score is not None:
        selected_rows = selected_rows[selected_rows['政治'] <= int(politics_score)]

    if english_score is not None:
        selected_rows = selected_rows[selected_rows['英语'] <= int(english_score)]

    if user_score is not None:
        lower_bound = user_score - float_value
        upper_bound = user_score + float_value
        selected_rows = selected_rows[(selected_rows['总分'] >= lower_bound) & (selected_rows['总分'] <= upper_bound)]

    school_name = entry_school.get().strip()
    if school_name and user_major:
        selected_rows = selected_rows[
            (selected_rows['学院名称'].str.contains(school_name)) & (selected_rows['专业名称'].str.contains(user_major))]
    else:
        if school_name:
            selected_rows = selected_rows[selected_rows['学院名称'].str.contains(school_name)]
        elif user_major:
            selected_rows = selected_rows[selected_rows['专业名称'].str.contains(user_major)]

    if academic_var.get() and professional_var.get():
        pass
    elif academic_var.get():
        selected_rows = selected_rows[selected_rows['硕士类型'] == '学术型硕士']
    elif professional_var.get():
        selected_rows = selected_rows[selected_rows['硕士类型'] == '专业型硕士']

    selected_rows['差距'] = abs(selected_rows['总分'] - user_score) if user_score is not None else 0
    selected_rows = reset_and_sort_index(selected_rows)

    result_table.delete(*result_table.get_children())

    for _, row in selected_rows.iterrows():
        values = [str(row[col_name]) for col_name in df.columns]
        result_table.insert("", tk.END, values=values)
    
    # 动态调整表格高度
    num_rows = len(selected_rows)
    table_height = min(num_rows, 10) * 20 + 25
    result_table.config(height=table_height)

    result_table.yview_moveto(0)
    

def show_menu(event):
    # 获取当前选中的行
    selected_item = result_table.selection()[0]

    # 创建右键菜单
    menu = Menu(result_table, tearoff=0)
    menu.add_command(label="学校官网", command=lambda: showinfo("学校官网", "学校风评信息"))
    menu.add_command(label="招生简章", command=lambda: showinfo("招生简章", "招生简章信息"))
    menu.add_command(label="学校风评", command=lambda: showinfo("学校风评", "学校风评信息"))
    menu.add_command(label="风景照", command=lambda: showinfo("风景照", "风景照信息"))
    menu.add_command(label="食堂评价", command=lambda: showinfo("食堂评价", "食堂评价信息"))
    menu.add_command(label="校友现状", command=lambda: showinfo("校友现状", "校友现状信息"))

    # 显示右键菜单
    menu.post(event.x_root, event.y_root)

def show_campus_website():
    # 获取当前选中的行
    selected_item = result_table.selection()[0]

    # 获取学校名称
    school_name = result_table.item(selected_item)["values"][0]

    # 显示学校官网信息
    showinfo("学校官网", f"{school_name} 的学校官网信息")
    
def show_Enrollment_Guide():
    # 获取当前选中的行
    selected_item = result_table.selection()[0]

    # 获取学校名称
    school_name = result_table.item(selected_item)["values"][0]

    # 显示招生简章信息
    showinfo("招生简章", f"{school_name} 的招生简章信息")

def show_School_style_evaluation():
    # 获取当前选中的行
    selected_item = result_table.selection()[0]

    # 获取学校名称
    school_name = result_table.item(selected_item)["values"][0]

    # 显示学校风评信息
    showinfo("学校风评", f"{school_name} 的学校风评信息")

def show_landscape_photography():
    # 获取当前选中的行
    selected_item = result_table.selection()[0]

    # 获取学校名称
    school_name = result_table.item(selected_item)["values"][0]

    # 显示风景信息
    showinfo("风景照", f"{school_name} 的风景照信息")
    
def show_Canteen_evaluation():
    # 获取当前选中的行
    selected_item = result_table.selection()[0]

    # 获取学校名称
    school_name = result_table.item(selected_item)["values"][0]

    # 显示食堂评价信息
    showinfo("食堂评价", f"{school_name} 的食堂评价信息")
    
def show_Alumni_status():
    # 获取当前选中的行
    selected_item = result_table.selection()[0]

    # 获取学校名称
    school_name = result_table.item(selected_item)["values"][0]

    # 显示校友现状信息
    showinfo("校友现状", f"{school_name} 的校友现状信息")


    
# 清空所有内容
def clear_all():
    entry_score.delete(0, 'end')
    entry_politics.delete(0, 'end')
    entry_english.delete(0, 'end')
    entry_float.delete(0, 'end')
    entry_major.delete(0, 'end')
    entry_school.delete(0, 'end')
    filter_var.set(False)
    filter_normal_var.set(False)
    filter_medical_var.set(False)
    filter_military_var.set(False)
    academic_var.set(False)
    professional_var.set(False)

# 选择地区按钮回调函数
def choose_region():
    global selected_regions

    def confirm_selection():
        # 将用户选择的地区添加到selected_regions列表中
        global selected_regions
        selected_regions = [regions[i] for i, var in enumerate(checkbox_vars) if var.get()]
        if len(selected_regions) > 3:
            selected_region_var.set("当前勾选的地区为：" + ", ".join(selected_regions[:3]) + "...")
        else:
            selected_region_var.set("当前勾选的地区为：" + ", ".join(selected_regions))
        window.destroy()

    def select_all():
        for var in checkbox_vars:
            var.set(True)

    def clear_all():
        for var in checkbox_vars:
            var.set(False)

    window = tk.Toplevel()
    window.title("选择地区")
    window.geometry("400x250")

    # 读取配置文件
    with open("config.json", "r",encoding = 'utf-8') as f:
        config_data = json.load(f)
    regions = list(config_data.keys())

    checkbox_vars = []

    for i, region in enumerate(regions):
        var = tk.BooleanVar(value=False)
        checkbox_vars.append(var)
        checkbutton = tk.Checkbutton(window, text=region, variable=var)
        checkbutton.grid(row=i // 4, column=i % 4, sticky="w")

    button_frame = tk.Frame(window)
    button_frame.grid(row=0, column=len(regions) // 4 + 1, rowspan=len(regions) // 4 + 1, padx=10)

    select_all_button = tk.Button(button_frame, text="我都可以", command=select_all)
    select_all_button.pack(side="top", pady=10)

    clear_button = tk.Button(button_frame, text="清空", command=clear_all)
    clear_button.pack(side="top", pady=10)

    confirm_button = tk.Button(button_frame, text="确认", command=confirm_selection)
    confirm_button.pack(side="top", pady=10)

    selected_label = tk.Label(window, text="")
    selected_label.grid(row=len(regions) // 4 + 2, column=0, columnspan=len(regions) // 4 + 1, pady=10)

    window.mainloop()

    return selected_regions



# 读取配置文件
def read_config_file():
    with open("config.json", "r",encoding = 'utf-8') as f:
        config_data = json.load(f)
    return config_data

# 选择的地区列表
selected_regions = []

# 读取配置文件
config_data = read_config_file()

# 读取数据文件并合并为一个数据框
dfs = []
for region, file_path in config_data.items():
    df = pd.read_excel(file_path)
    dfs.append(df)
df = pd.concat(dfs)

# 创建GUI窗口
window = tk.Tk()
window.title("考研个性化推荐")
window.geometry("1100x600")


# 创建输入框部分的容器
input_container = tk.Frame(window)
input_container.pack(side=tk.TOP, pady=10)

# 创建选择地区按钮
choose_region_button = tk.Button(input_container, text="选择地区", command=choose_region)
choose_region_button.grid(row=0, column=0)

# 创建显示当前勾选地区的标签
selected_region_var = tk.StringVar()
selected_label = tk.Label(input_container, textvariable=selected_region_var)
selected_label.grid(row=1, column=0)
selected_label.config(font=("Arial", 9))

# 创建标签、输入框和按钮
label_score = tk.Label(input_container, text="你模拟的总分：")
label_score.grid(row=0, column=1)
entry_score = tk.Entry(input_container)
entry_score.grid(row=0, column=2)
entry_score.config(width=10)

# 创建政治分数输入框
label_politics = tk.Label(input_container, text="政治分数：")
label_politics.grid(row=0, column=3)
entry_politics = tk.Entry(input_container)
entry_politics.grid(row=0, column=4)
entry_politics.config(width=4)

# 创建英语分数输入框
label_english = tk.Label(input_container, text="英语分数：")
label_english.grid(row=0, column=5)
entry_english = tk.Entry(input_container)
entry_english.grid(row=0, column=6)
entry_english.config(width=4)

# 创建学校类型过滤选项的容器
filter_container = tk.Frame(window)
filter_container.pack(side=tk.TOP)

# 创建过滤选项的标签和复选框
filter_var = tk.BooleanVar(value=True)
filter_checkbox = tk.Checkbutton(filter_container, text="过滤师范、医科、军校", variable=filter_var)
filter_checkbox.grid(row=0, column=0)

# 创建只看师范的勾选框
filter_normal_var = tk.BooleanVar()
filter_normal_checkbox = tk.Checkbutton(filter_container, text="只看师范", variable=filter_normal_var)
filter_normal_checkbox.grid(row=0, column=1)

# 创建只看医科的勾选框
filter_medical_var = tk.BooleanVar()
filter_medical_checkbox = tk.Checkbutton(filter_container, text="只看医科", variable=filter_medical_var)
filter_medical_checkbox.grid(row=0, column=2)

# 创建只看军校的勾选框
filter_military_var = tk.BooleanVar()
filter_military_checkbox = tk.Checkbutton(filter_container, text="只看军校", variable=filter_military_var)
filter_military_checkbox.grid(row=0, column=3)

# 创建硕士类型过滤选项的容器
master_type_container = tk.Frame(window)
master_type_container.pack(side=tk.TOP)

# 创建只看学术型硕士的复选框
academic_var = tk.BooleanVar()
academic_checkbox = tk.Checkbutton(master_type_container, text="只看学术型硕士", variable=academic_var)
academic_checkbox.grid(row=0, column=0)

# 创建只看专业型硕士的复选框
professional_var = tk.BooleanVar()
professional_checkbox = tk.Checkbutton(master_type_container, text="只看专业型硕士", variable=professional_var)
professional_checkbox.grid(row=0, column=1)

# 浮动分数输入框
label_float = tk.Label(input_container, text="总分上下浮动数值：")
label_float.grid(row=0, column=7)
entry_float = tk.Entry(input_container)
entry_float.grid(row=0, column=8)
entry_float.insert(0, "20")
entry_float.config(width=5)

# 查询按钮
find_button = tk.Button(input_container, text="查询", command=find_closest_scores_with_major)
find_button.grid(row=0, column=9, padx=10)

# 清空所有按钮
clear_button = tk.Button(input_container, text="清空所有", command=clear_all)
clear_button.grid(row=0, column=10, padx=10)

def export_to_excel():
    # 获取当前表格中的数据
    global df
    selected_rows = []
    for item in result_table.get_children():
        values = result_table.item(item)["values"]
        selected_rows.append(values)

    # 将数据转换为 pandas DataFrame
    df = pd.DataFrame(selected_rows, columns=list(df.columns))

    # 保存为 Excel 文件
    file_path = tk.filedialog.asksaveasfilename(
        defaultextension=".xlsx",
        filetypes=[("Excel 文件", "*.xlsx")],
        title="保存筛选结果")

    if file_path:
        df.to_excel(file_path, index=False)
        tk.messagebox.showinfo("成功", "数据已保存到 Excel 文件中。")
# 创建导出按钮
export_button = tk.Button(input_container, text="导出", command=export_to_excel)
export_button.grid(row=0, column=11, padx=10)

# 创建单独的学校输入框
label_school = tk.Label(window, text="如果你有心仪的学校：")
label_school.pack()
entry_school = tk.Entry(window)
entry_school.pack()

# 创建单独的专业输入框
label_major = tk.Label(window, text="如果你有喜欢的专业：")
label_major.pack()
entry_major = tk.Entry(window)
entry_major.pack()

# 创建一个容器来包含表格和竖直滚动条
table_container = tk.Frame(window)
table_container.pack(fill="both", expand=True)

# 创建查询结果展示表格
result_table = ttk.Treeview(table_container, columns=list(df.columns), show="headings",
                            selectmode="browse")

# 添加列标题
for col_name in df.columns:
    result_table.heading(col_name, text=col_name)

# 设置初始列宽度
target_columns = ["总分", "政治", "英语", "专业课一", "专业课二"]
for i, col_name in enumerate(df.columns):
    if col_name == "备注":
        result_table.column(col_name, width=150)
    elif col_name == "专业名称":
        result_table.column(col_name, width=185)
    elif col_name == "硕士类型":
        result_table.column(col_name, width=75)
    elif col_name == "专业代码":
        result_table.column(col_name, width=65)
    elif col_name in target_columns:
        result_table.column(col_name, width=60)
    else:
        result_table.column(col_name, width=90)

# 将数据插入表格中
result_table_height = 150  # 调整此值以增加或减少表格的高度
result_table.place(x=20, y=230, width=1060, height=320, anchor="nw")

# 设置表格填充整个窗口
result_table.pack(side="left", fill="both", expand=True)

# 创建垂直滚动条
y_scrollbar = Scrollbar(table_container, orient="vertical", command=result_table.yview)
y_scrollbar.pack(side="right", fill="y")

result_table.bind("<Button-3>", show_menu)




window.mainloop()
